In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import taichi as ti
import numpy as np
import math
from engine.mpm_solver import MPMSolver
import argparse
import os




out_dir = 'out_jupyter'

ti.init(arch=ti.cuda)  # Try to run on GPU

gui = ti.GUI("Taichi Elements", res=512, background_color=0x112F41)

mpm = MPMSolver(res=(256, 256), E_scale=0.001)
mpm2 = MPMSolver(res=(256,256), E_scale=0.5)
mpm.set_gravity([0, -9.81])

E1 = 1e6
E2 = 1e3

mpm.add_spikes(
            sides=4,
            center=[0.15,0.31],
            velocity=[0.1,0],
            radius=0.1,
            width = 0.02,
            color=0xFFAAAA,
            sample_density=4,
            material=mpm.material_elastic,
            E=E1,
            nu=0.2)
mpm.add_cube(
    lower_corner=[0.,0],
    cube_size=[0.1,0.1],
    velocity=[0, 0],
    sample_density=1,
    material=mpm.material_elastic,
    E=E2,
    nu=0.2
)
mpm.add_cube(
    lower_corner=[0.9,0],
    cube_size=[0.1,0.1],
    velocity=[0, 0],
    sample_density=1,
    material=mpm.material_elastic,
    E=E2,
    nu=0.2
)
mpm.add_cube(
    lower_corner=[0.,0.1],
    cube_size=[1,0.1],
    velocity=[0, 0],
    sample_density=1,
    material=mpm.material_elastic,
    E=E1,
    nu=0.2
)
for frame in range(50):
    mpm.step(8e-3)
    particles = mpm.particle_info()
    print(mpm.fan_center, mpm.t, mpm.omega)
    gui.circles(particles['position'], radius=1.5, color=particles['color'])
    gui.show(f'{out_dir}/{frame:06d}.png')

[Taichi] Starting on arch=cuda
[0.15062751 0.30969664] 0.008000000000000018 0.0
[0.15127598 0.3087598 ] 0.01599999999999917 0.0
[0.15194902 0.30718577] 0.023999999999998325 0.0
[0.15264305 0.3049687 ] 0.03199999999999748 0.0
[0.15335312 0.30210754] 0.039999999999996635 0.0
[0.15407571 0.2986034 ] 0.04799999999999579 0.0
[0.15480919 0.29445487] 0.05599999999999494 0.0
[0.15555021 0.28966036] 0.06399999999999544 0.0
[0.15723933 0.28538817] 0.0720000000000017 0.0
[0.16298561 0.28231296] 0.08000000000000797 0.0
[0.16817556 0.27854428] 0.08800000000001423 0.0
[0.17304367 0.27480257] 0.0960000000000205 0.0
[0.1776323  0.27157536] 0.10400000000002677 0.0
[0.18207781 0.2687309 ] 0.11200000000003303 0.0
[0.18634309 0.26617923] 0.1200000000000393 0.0
[0.190531   0.26391223] 0.12800000000004022 0.0
[0.19459024 0.2616763 ] 0.13600000000003226 0.0
[0.19859841 0.25936997] 0.1440000000000243 0.0
[0.20254953 0.25698987] 0.15200000000001634 0.0
[0.20649593 0.2544515 ] 0.16000000000000839 0.0
[0.2104613

In [39]:

ti.init()

field = ti.Vector.field(3,dtype=float,shape=(1,1,1))

# @ti.kernel
def loop():
    for i,j,k in ti.ndrange(1,1,1):
        # field[i,j,k] = i+j+k
        print(field[i,j,k])

loop()

[Taichi] Starting on arch=x64
[0. 0. 0.]


In [3]:
ti.reset()

In [1]:
import numpy as np

def rolling_gaussian_generator(velocity, frequency, node_x_locs, N=5, resolution=200):
    time_to_center = node_x_locs / velocity

    fc = frequency
    t_width = N / fc  # half-width of time range

    t = np.linspace(-t_width, t_width, resolution, endpoint=False)
    _, _, e = gausspulse(t, fc=fc)

    t_pad = np.array([-t_width - 1e-16, t_width + 1e-16])
    e_pad = np.array([0, 0])

    t = np.concatenate([t_pad[:1], t, t_pad[1:]])
    e = np.concatenate([e_pad[:1], e, e_pad[1:]])

    xvalues = t + time_to_center[:, np.newaxis]
    fxvalues = e
    return xvalues, fxvalues

def gausspulse(t, fc=1000, bw=0.5, bwr=-6):
    """
    Return a Gaussian modulated sinusoid:

        ``exp(-a t^2) exp(1j*2*pi*fc*t).``

    Parameters
    ----------
    t : ndarray
        Input array.
    fc : float, optional
        Center frequency (e.g. Hz).  Default is 1000.
    bw : float, optional
        Fractional bandwidth in frequency domain of pulse (e.g. Hz).
        Default is 0.5.
    bwr : float, optional
        Reference level at which fractional bandwidth is calculated (dB).
        Default is -6.

    Returns
    -------
    yI : ndarray
        Real part of signal.
    yQ : ndarray
        Imaginary part of signal.
    yenv : ndarray
        Envelope of signal.
    """

    # Convert t to a jnp array if it's not already
    t = np.asarray(t)

    # exp(-a t^2) <->  sqrt(pi/a) exp(-pi^2/a * f^2)  = g(f)
    ref = np.power(10.0, bwr / 20.0)
    
    # pi^2/a * fc^2 * bw^2 /4=-log(ref)
    a = -(np.pi * fc * bw) ** 2 / (4.0 * np.log(ref))

    yenv = np.exp(-a * t * t)
    yI = yenv * np.cos(2 * np.pi * fc * t)
    yQ = yenv * np.sin(2 * np.pi * fc * t)

    return yI, yQ, yenv

# external forces
velocity = 4
frequency = 5
node_x_locs = np.arange(5)
xvalues, fxvalues = rolling_gaussian_generator(velocity, frequency, node_x_locs, resolution=5)
fxvalues.shape

(7,)

In [7]:
import taichi as ti

ti.init(arch=ti.cuda)


dim = 2
num_nodes = 9
t_intervals = 5

node_ids_fext = range(5)

f_ext = ti.Vector.field(t_intervals, dtype=ti.f32, shape=(num_nodes,dim+1))

# @ti.kernel
# def assign_value():
# test[0] = [1.0] 

for i, node in enumerate(node_ids_fext):
    f_ext[node,0] = xvalues[i]
    f_ext[node,2] = fxvalues

# Access the value
print(f_ext[2,0])
print(f_ext[2,2])


def interpolate(x, xvalues, fxvalues):
    assert len(xvalues) == len(fxvalues)
    # Check if x is outside the range of xvalues
    if x < min(xvalues) or x > max(xvalues):
        raise ValueError("x is outside the range of xvalues")

    # Find the two closest x-values to x
    i = 0
    while x > xvalues[i + 1]:
        i += 1
        
    # Linear interpolation formula
    x0, x1 = xvalues[i], xvalues[i + 1]
    fx0, fx1 = fxvalues[i], fxvalues[i + 1]

    interpolated_value = fx0 + (fx1 - fx0) * (x - x0) / (x1 - x0)

    return interpolated_value

interpolate(-0.1, f_ext[2,0], f_ext[2,2])

[Taichi] Starting on arch=cuda
[-0.5        -0.5        -0.1         0.30000001  0.69999999]
[0.00000000e+00 2.01636235e-10 3.23329004e-04 4.09432918e-01
 4.09432918e-01]


0.0003233305285110086